# DuraMAT Workshop Live Demo - Geospatial analysis

![PVDeg Logo](../PVD_logo.png)

**Steps:**
1. Initialize weather data into xarray
2. Calculate installation standoff for New Mexico
3. Plot results

**Note:** This demo requires access to NREL's HPC system for full functionality. When running outside NREL, we'll provide informative messages about the required connections.

**Xarray: multi-dimensional data frame**

![Xarray](./images/xarray.webp)

In [ ]:
import pandas as pd
import pvdeg
import numpy as np
import warnings

In [ ]:
# This information helps with debugging and getting support :)
import sys
import platform

print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("pvdeg version ", pvdeg.__version__)

## 1 Start distributed compute cluster - DASK

In [ ]:
pvdeg.geospatial.start_dask();

In [ ]:
# Get weather data
weather_db = "NSRDB"

weather_arg = {
    "satellite": "Americas",
    "names": 2022,
    "NREL_HPC": True,
    "attributes": [
        "air_temperature",
        "wind_speed",
        "dhi",
        "ghi",
        "dni",
        "relative_humidity",
    ],
}

# Set a flag to track if geospatial data is available
geospatial_data_available = True

try:
    weather_ds, meta_df = pvdeg.weather.get(weather_db, geospatial=True, **weather_arg)
    print("Successfully loaded geospatial weather data")
except ConnectionError as e:
    print(f"ConnectionError: {e}")
    print("Geospatial weather data requires connection to NREL HPC resources.")
    print("This demo requires access to NREL's internal systems.")
    geospatial_data_available = False
    weather_ds, meta_df = None, None
except Exception as e:
    print(f"Error loading geospatial weather data: {e}")
    print("Continuing without geospatial data.")
    geospatial_data_available = False
    weather_ds, meta_df = None, None

In [ ]:
if geospatial_data_available:
    weather_ds
else:
    print("Weather dataset not available - requires NREL HPC connection")

In [ ]:
if geospatial_data_available:
    meta_NM = meta_df[meta_df["state"] == "New Mexico"]
else:
    print("Metadata not available - requires successful weather data loading")
    meta_NM = None

In [ ]:
if geospatial_data_available:
    meta_NM_sub, gids_NM_sub = pvdeg.utilities.gid_downsampling(meta_NM, 4)
    weather_NM_sub = weather_ds.sel(gid=meta_NM_sub.index)
else:
    print("Cannot perform downsampling - requires successful weather data loading")
    meta_NM_sub, weather_NM_sub = None, None

In [ ]:
if geospatial_data_available:
    geo = {
        "func": pvdeg.standards.standoff,
        "weather_ds": weather_NM_sub,
        "meta_df": meta_NM_sub,
    }

    standoff_res = pvdeg.geospatial.analysis(**geo)
else:
    print("Cannot perform geospatial standoff analysis - requires successful weather data loading")
    standoff_res = None

In [ ]:
if geospatial_data_available:
    standoff_res
else:
    print("Standoff results not available - requires successful geospatial analysis")

In [ ]:
if geospatial_data_available:
    fig, ax = pvdeg.geospatial.plot_USA(
        standoff_res["x"],
        cmap="viridis",
        vmin=0,
        vmax=None,
        title="Minimum estimated air standoff to qualify as level 1 system",
        cb_title="Standoff (cm)",
    )
else:
    print("Cannot create standoff plot - requires successful geospatial analysis")

# Relative Humidity Example - Time dimension

In [ ]:
# State bar of new mexico: (35.16482, -106.58979)

weather_db = "NSRDB"
weather_id = (35.16482, -106.58979)  # NREL (39.741931, -105.169891)
weather_arg = {
    "satellite": "Americas",
    "names": 2022,
    "NREL_HPC": True,
    "attributes": [
        "air_temperature",
        "wind_speed",
        "dhi",
        "ghi",
        "dni",
        "relative_humidity",
    ],
}

# Set a flag to track if point weather data is available
point_data_available = True

try:
    weather_df, meta = pvdeg.weather.get(
        weather_db, weather_id, geospatial=False, **weather_arg
    )
    print("Successfully loaded point weather data")
except FileNotFoundError as e:
    print(f"FileNotFoundError: {e}")
    print("Point weather data requires access to NREL HPC file systems.")
    point_data_available = False
    weather_df, meta = None, None
except Exception as e:
    print(f"Error loading point weather data: {e}")
    print("Continuing without point weather data.")
    point_data_available = False
    weather_df, meta = None, None

In [ ]:
if point_data_available:
    RH_module = pvdeg.humidity.module(weather_df=weather_df, meta=meta)
else:
    print("Cannot calculate module humidity - requires successful weather data loading")
    RH_module = None

In [ ]:
if point_data_available:
    RH_module
else:
    print("Module humidity results not available - requires successful point weather data loading")

In [ ]:
if point_data_available:
    RH_module.plot(ls="--")
else:
    print("Cannot create humidity plot - requires successful module humidity calculation")

In [ ]:
if geospatial_data_available:
    geo = {
        "func": pvdeg.humidity.module,
        "weather_ds": weather_NM_sub,
        "meta_df": meta_NM_sub,
    }

    RH_module = pvdeg.geospatial.analysis(**geo)
else:
    print("Cannot perform geospatial humidity analysis - requires successful weather data loading")
    RH_module = None

In [ ]:
if geospatial_data_available:
    RH_module
else:
    print("Geospatial humidity results not available - requires successful geospatial analysis")

In [ ]:
# from matplotlib.animation import FuncAnimation
# from matplotlib.animation import PillowWriter
# import matplotlib.animation as animation
# import datetime
# ims = []
# for n in range(1, 13):
#     for i, np_t in enumerate(RH_module.time):
#         t = pd.Timestamp(np_t.values).time()
#         d = pd.Timestamp(np_t.values).day
#         m = pd.Timestamp(np_t.values).month
#         if m == n:
#             if d == 15:
#                 if t == datetime.time(12):
#                     fig, ax = pvdeg.geospatial.plot_USA(RH_module['RH_surface_outside'].sel(time=np_t),
#                             cmap='viridis', vmin=0, vmax=100,
#                             title=f'RH_surface_outside  - 2022-{m}-{d} 12:00',
#                             cb_title='Relative humidity (%)')
#                     plt.savefig(f'./images/RH_animation_{n}.png', dpi=600)

# import imageio
# ims = [imageio.imread(f'./images/RH_animation_{n}.png') for n in range(1, 13)]
# imageio.mimwrite(f'./images/RH_animation.gif', ims, format='GIF', duration=1000, loop=10)

![PVDeg Logo](./images/RH_animation.gif)